In [1]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, WDiscriminator
from off_manifolder import offmanifolder_maker
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit, GeodesicBridge
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset


import hydra
from omegaconf import DictConfig, OmegaConf
import wandb
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint


In [11]:
# cfg_main_ae_sweep_id = '8jfjcnrt'
cfg_main_ae_sweep_id = 'ycgsbnib'
cfg_main_disc_sweep_id = '1k78kgzf'
cfg_main_data_name = 'EB_phate'
cfg_main_weights_cycle = 0.
cfg_main_dimensions_latent = 2

In [12]:
# results_path = f'results/{cfg_main.data_name}/{cfg_main.dimensions_latent}'
# pathlib.Path(results_path).mkdir(parents=True, exist_ok=True)

wandb.login()
api = wandb.Api()

entity = "xingzhis"
project = "dmae"
sweep_id = cfg_main_ae_sweep_id
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

data_name = cfg_main_data_name
# run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
# run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == cfg_main_weights_cycle) & (df['dimensions.latent'] == cfg_main_dimensions_latent)]['run_id']
run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == cfg_main_weights_cycle) & (df['dimensions.latent'] == cfg_main_dimensions_latent) & (df['cfg/loss/weights/reconstr']==0.01) & (df['cfg/loss/weights/dist']==77.4)]['run_id']
assert len(run_ids) == 1
run_id = run_ids.iloc[0]
run = api.run(f"{entity}/{project}/{run_id}")
# run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
cfg.data.root = '../' + cfg.data.root
model = Autoencoder.load_from_checkpoint(ckpt_path)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
    zhh = model.encoder(xh)
zc = z.cpu().numpy()
# fig = scatter(zc, s=2, alpha=0.2, title='latent', filename=f'{results_path}/latent.html')
# wandb.log({"latent": wandb.Plotly(fig)})
# fig = scatter(zhh, s=2, alpha=0.2, title='latent_reembedd', filename=f'{results_path}/latent_reembedded.html')
# wandb.log({"latent_reembedded": wandb.Plotly(fig)})
# fig = scatter(x, s=2, alpha=0.2, title='data', filename=f'{results_path}/data.html')
# wandb.log({"data": wandb.Plotly(fig)})
# fig = scatter(xh, s=2, alpha=0.2, title='reconstruction', filename=f'{results_path}/reconstruction.html')
# wandb.log({"reconstruction": wandb.Plotly(fig)})

sweep_id = cfg_main_disc_sweep_id
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)
run_ids = df[(df['loss.weights.pos1'] == 1.)][['run_id']]
assert len(run_ids) == 1
run_id = run_ids.iloc[0]
run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
cfg.data.root = '../' + cfg.data.root
wd = WDiscriminator.load_from_checkpoint(ckpt_path)
data_ = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    wd.eval()
    x_disc = torch.tensor(data_['data'], dtype=torch.float32, device=wd.device)
    probab = wd(x_disc).flatten()
    z = model.encoder(x_disc)

# # title = f'{data_name} {noise_type} {noise_level} {dist_mask}'
# scatter(zc, data['mask_x'][data['is_train'].astype(bool)], s=2, alpha=0.2).show()
# scatter(zhh, data['mask_x'][data['is_train'].astype(bool)], s=2, alpha=0.2).show()
# scatter(x_disc, data['mask_x'][data['is_train'].astype(bool)], s=2, alpha=0.2, title=f'{data_name}: input').show()
# scatter(x_disc, probab, s=2, alpha=0.2, title=f'{data_name}: probability').show()#, filename='hsphere_disc_prob.html').show()
# scatter(x_disc, probab > 0.5, s=2, alpha=0.2, title=f'{data_name}: classification').show()

# title = f'[RECON] {title}'
# if savefolder is not None:
#     filename = f"{savefolder}/{title1}.html"
# scatter(xh, data['mask_x'][data['is_train'].astype(bool)], s=2, alpha=0.2).show()

max_prob = probab.max()


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.



In [13]:
scatter(zc, data['colors'], s=3, white_bkgrnd=True, ticks=False, rescale=True).show()

In [14]:
np.unique(data['colors'])

array([0, 1, 2, 3, 4])

In [15]:
data['colors']

array([1, 2, 2, ..., 0, 0, 3])

In [16]:
# import numpy as np
# np.random.seed(23)
# import numpy as np
# colors = data['colors']
# num_samples = 5
# # start_idx = np.random.choice(len(colors), 1, replace=False)
# # start_idx = start_idx.repeat(num_samples)
# start_idx = np.random.choice(len(colors), num_samples, replace=False)
# end_idx = []

# for idx in start_idx:
#     valid_indices = np.where(colors > colors[idx])[0]
    
#     if len(valid_indices) > 0:
#         end_idx.append(np.random.choice(valid_indices))
#     else:
#         end_idx.append(-1)  # or you can re-sample the start_idx or take other action

# end_idx = np.array(end_idx)

In [17]:
scatter(zc, s=1, title='Geodesic latent').show()

In [94]:
start_idx = []
end_idx = []
id0 = np.where(zc[:,1]>3)[0][0]
id1 = np.where((zc[:,0]<-1.3)&(zc[:,1]<0.1)&(zc[:,1]>-.1))[0][0]
start_idx.append(id0)
end_idx.append(id1)
id2 = np.where((zc[:,0]>-1)&(zc[:,0]<-0.9)&(zc[:,1]>-.6)&(zc[:,1]<-.5))[0][0]
start_idx.append(id1)
end_idx.append(id2)
start_idx.append(id1)
end_idx.append(id2)
id3 = np.where((zc[:,0]>.1)&(zc[:,0]<.2)&(zc[:,1]>-.6)&(zc[:,1]<-.5))[0][0]
start_idx.append(id1)
end_idx.append(id3)
id4 = np.where((zc[:,0]>.1)&(zc[:,0]<.2)&(zc[:,1]>-1.3)&(zc[:,1]<-1.2))[0][0]
start_idx.append(id2)
end_idx.append(id4)
id5 = np.where((zc[:,0]>1.8)&(zc[:,0]<1.9)&(zc[:,1]>-1)&(zc[:,1]<-.9))[0][0]
start_idx.append(id4)
end_idx.append(id5)
start_idx.append(id3)
end_idx.append(id5)

In [95]:
xbatch = x[start_idx]
xendbatch = x[end_idx]

In [96]:
with torch.no_grad():
    z0 = model.encoder(xbatch)
    z1 = model.encoder(xendbatch)

In [97]:
# fig = scatter(zc, data['colors'], s=3)
fig = scatter(zc, s=3)
fig = scatter(z0.detach().cpu().numpy(), s=10, fig=fig)
fig = scatter(z1.detach().cpu().numpy(), s=10, fig=fig, title='Geodesic latent', white_bkgrnd=True, ticks=False)
fig.show()

In [101]:
ids = torch.zeros((xbatch.size(0),1))
# ids = torch.eye((xbatch.size(0)))

dataset = TensorDataset(xbatch, xendbatch, ids)
dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
wd = wd.to(device)
# med = probab.quantile(1-data['mask_x'].mean()).to(device)
# discriminator = discriminator.to(device)
for param in model.encoder.parameters():
    param.requires_grad = False
for param in wd.parameters():
    param.requires_grad = False
enc_func = lambda x: model.encoder(x)
# disc_func = lambda x: wd(x).flatten()
# disc_func_pen = lambda x: 1 - (max_prob - disc_func(x))
# discriminator_func_for_grad = lambda x: wd(x)
# discriminator_func_for_grad = lambda x: discriminator.positive_proba(x).reshape(-1,1)
disc_func = lambda x: (wd(x).flatten()-probab.min())/(probab.max()-probab.min())
# disc_func = lambda x: (torch.clamp(wd(x).flatten(), max=med) - probab.min())/(med-probab.min())
# disc_func_pen = disc_func
# discriminator_func_for_grad = lambda x: (wd(x)-probab.min())/(probab.max()-probab.min())
ofm = offmanifolder_maker(enc_func, disc_func, disc_factor=4., max_prob=max_prob)
gbmodel = GeodesicBridgeOverfit(
    func=ofm,
    # func = enc_func,
    # discriminator_func=disc_func_pen,
    # discriminator_func_for_grad=discriminator_func_for_grad,
    input_dim=x.size(1), 
    hidden_dim=64, 
    scale_factor=1, 
    symmetric=True, 
    num_layers=3, 
    n_tsteps=100, 
    lr=1e-3, 
    weight_decay=1e-3,
    discriminator_weight=0.,
    discriminator_func_for_grad_weight=0.,
    id_dim=1,
    id_emb_dim=1,
    density_weight=0.,
    length_weight=1.,
)

# if cfg_main.use_wandb:
#     logger = WandbLogger()
#     checkpoint_dir = wandb.run.dir  # Use wandb's run directory for saving checkpoints

# else:
#     logger = TensorBoardLogger(save_dir=os.path.join(results_path))
#     checkpoint_dir = results_path  # Use a local directory for saving checkpoints


gbmodel.lr=1e-3
trainer = pl.Trainer(
    # logger=logger,
    max_epochs=400,
    log_every_n_steps=20,
    accelerator='cuda',
)
trainer.fit(gbmodel, dataloader)
# trainer.save_checkpoint(f"{checkpoint_dir}/gbmodel.ckpt")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | cc   | CondCurveOverfit | 51.4 K
------------------------------------------
51.4 K    Trainable params
0         Non-trainable params
51.4 K    Total params
0.206     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=400` reached.


In [102]:
checkpoint_dir = '.'
results_path = '.'
trainer.save_checkpoint(f"{checkpoint_dir}/gbmodel.ckpt")

In [103]:
x0, x1, ids = next(iter(dataloader))
x0 = x0.to(device)
x1 = x1.to(device)
ids = ids.to(device)
x0 = x0[:10]
x1 = x1[:10]
ids = ids[:10]
x = x.to(device)
gbmodel = gbmodel.to(device)
with torch.no_grad():
    xhat = gbmodel(x0, x1, gbmodel.ts.to(device), ids)

xshape = xhat.shape
xflatten = xhat.flatten(0,1)
with torch.no_grad():
    zhat_flatten = model.encoder(xflatten)
    zhat = zhat_flatten.reshape(xshape[0], xshape[1], -1)
    z0 = model.encoder(x0)
    z1 = model.encoder(x1)

    fig = scatter(x.cpu().numpy(), data['colors'], s=3, white_bkgrnd=True, ticks=False)
    fig = scatter(x0.detach().cpu().numpy(), s=10, fig=fig)
    fig = scatter(x1.detach().cpu().numpy(), s=10, fig=fig)
    fig = trajectories(xhat.detach().cpu().numpy(), s=5, fig=fig, title='Geodesic', filename='geodesic_ofm.html')
    fig.show()
# if cfg_main.use_wandb:
#     wandb.log({"geodesic": wandb.Plotly(fig)})

fig = scatter(zc, data['colors'], s=3)
fig = scatter(z0.detach().cpu().numpy(), s=10, fig=fig)
fig = scatter(z1.detach().cpu().numpy(), s=10, fig=fig)
fig = trajectories(zhat.detach().cpu().numpy(), s=1, fig=fig, title='Geodesic latent', filename=f'{results_path}/geodesic_latent.html', white_bkgrnd=True, ticks=False)
fig.show()
# if cfg_main.use_wandb:
#     wandb.log({"geodesic_latent": wandb.Plotly(fig)})

with torch.no_grad():
    xconc = xhat.flatten(0, 1)
    probs = disc_func(xconc)
fig = scatter(xconc.cpu().numpy(), probs.cpu().numpy(), s=2, title='geodesic on-mfd probability', filename=f'{results_path}/geodesic_on_mfd_prob.html')
fig.show()